<a href="https://colab.research.google.com/github/angzhifan/Importance-Weighted-Autoencoders/blob/main/IWAE_table2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#IWAE experiment 2
#Author: Angzhi Fan

import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import numpy as np
import math

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# VAE or IWAE with one layer
class VAE_1(nn.Module):
    
    def __init__(self, t):
        super(VAE_1, self).__init__()
        self.k = t
        self.fc1 = nn.Linear(28*28, 200)
        self.fc2 = nn.Linear(200, 200)
        self.fc3_mu = nn.Linear(200, 50)
        self.fc3_sigma = nn.Linear(200, 50)
        self.fc4 = nn.Linear(50, 200)
        self.fc5 = nn.Linear(200, 200)
        self.fc6 = nn.Linear(200,28*28)
        
        
    def forward(self, x):
        x = x.view(-1,1,28*28)
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        mu = self.fc3_mu(x).view(-1,1,50)
        log_sigma = self.fc3_sigma(x).view(-1,1,50)
        eps = torch.randn_like(mu.repeat(1,self.k,1))
        x = mu.repeat(1,self.k,1) + torch.exp(log_sigma.repeat(1,self.k,1))*eps
        x = torch.tanh(self.fc4(x))
        x = torch.tanh(self.fc5(x))
        x = self.fc6(x)
        return x, mu, log_sigma, eps
    

cpu


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def load_data():
    train_file = '/content/drive/My Drive/dataset/BinaryMNIST/binarized_mnist_train.amat'
    valid_file = '/content/drive/My Drive/dataset/BinaryMNIST/binarized_mnist_valid.amat'
    test_file = '/content/drive/My Drive/dataset/BinaryMNIST/binarized_mnist_test.amat'
    mnist_train = np.concatenate([np.loadtxt(train_file),np.loadtxt(valid_file)])
    mnist_test = np.loadtxt(test_file)
    return mnist_train, mnist_test

mnist_train, mnist_test = load_data()
print(mnist_train.shape)
print(mnist_test.shape)

(60000, 784)
(10000, 784)


In [5]:
def test_function_1(net):
    testloader = torch.utils.data.DataLoader(mnist_test, batch_size=20,shuffle=False)
    nll = 0.0
    A_u = torch.zeros(50)
    for i,data in enumerate(testloader, 0):
      with torch.no_grad():
        test = data.view(-1,1,28*28).to(device)
        output = net(test.float())
        
        # stochastic layer
        eps = torch.randn_like(output[2].repeat(1,5000,1))
        h = output[1].repeat(1,5000,1) + torch.exp(output[2].repeat(1,5000,1))*eps

        # output of x using the new epsilon
        output_x = torch.tanh(net.fc4(h))
        output_x = torch.tanh(net.fc5(output_x))
        output_x = net.fc6(output_x)
        log_prob_condi = torch.sum(output_x*test.repeat(1,5000,1)-torch.log(1+torch.exp(output_x)), 2)

        # log weights, unnormalized
        log_weights = log_prob_condi-(h*h).sum(2)/2+(eps*eps).sum(2)/2+output[2].repeat(1,5000,1).sum(2)

        # estimate log likelihood using L_5000
        L_5000 = log_weights.max(1)[0].mean()+torch.log(torch.exp(log_weights
                        -log_weights.max(1)[0].view(-1,1)).mean(1)).mean()
        nll -= L_5000.item()
        A_u += output[1].view(-1,50).var(0).cpu()
    return nll/500, sum(A_u.detach().numpy()/500>0.01)

print("Finished loading test function 1")

Finished loading test function 1


In [ ]:
#Initialize it by a stage1_mod and train it using stage2_mod objective function
stage2_mod = 'iwae'
layer = 1
batch_size = 20
k = 1
continued = 121

if stage2_mod == 'iwae':
    #this index_vec will be used in the training of iwae
    index_vec = torch.tensor([i*k for i in range(batch_size)]).to(device)
    stage1_mod = 'vae'
elif stage2_mod == 'vae':
    stage1_mod = 'iwae'
else:
    raise Exception("Invalid Mode")

net = VAE_1(k)
#net.load_state_dict(torch.load('/content/drive/My Drive/IWAE_VAE/model/'+stage1_mod+'_net'+'_layer'+str(layer)+'_k'+str(k)+'_'+'.pth',map_location=torch.device('cpu')))
net.load_state_dict(torch.load('/content/drive/My Drive/IWAE_VAE/model/stage1_modvaestage2_modiwae_net_layer1_k1_162.pth',map_location=torch.device('cpu')))





with open('/content/drive/My Drive/IWAE_VAE/outfile_stage1_mod'+stage1_mod+'stage2_mod'+stage2_mod+'_layer'+str(layer)+'_k'+str(k)+'_'+'.txt', 'w') as outfile:
    outfile.write('output of the code '+'\n'+'author:Angzhi Fan fana@uchicago.edu'+'\n')
    
start = time.time()
trainloader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, 
                                         shuffle=True, num_workers=2)
decay_num = sum(np.array([1, 4, 13, 40, 121, 364, 1093, 3280])<=continued)
learning_rate = 0.001/10**(decay_num/7)
optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate
                             , betas=(0.9, 0.999), eps=1e-04)
for epoch in range(continued, continued+81):
    if epoch in [continued+80, continued+240]:
        PATH = '/content/drive/My Drive/IWAE_VAE/model/stage1_mod'+stage1_mod+'stage2_mod'+stage2_mod+'_net'+'_layer'+str(layer)+'_k'+str(k)+'_'+str(epoch)+'.pth'
        torch.save(net.state_dict(), PATH)
        with open('/content/drive/My Drive/IWAE_VAE/outfile_stage1_mod'+stage1_mod+'stage2_mod'+stage2_mod+'_layer'+str(layer)+'_k'+str(k)+'_'+'.txt', 'a') as outfile:
            if layer == 1:
                nll, A_u = test_function_1(net)
            else:
                nll, A_u = test_function_2(net)
            print('NLL:', nll, 'active units:', A_u)
            outfile.write('test average (NLL):'+str(nll)+'\n')
            outfile.write('test average (active units):'+str(A_u)+'\n')
            outfile.write('learning rate decay'+'\n')
        print("learning rate=", learning_rate)
    if epoch in [1, 4, 13, 40, 121, 364, 1093, 3280]:
        PATH = '/content/drive/My Drive/IWAE_VAE/model/stage1_mod'+stage1_mod+'stage2_mod'+stage2_mod+'_net'+'_layer'+str(layer)+'_k'+str(k)+'_'+str(epoch)+'.pth'
        torch.save(net.state_dict(), PATH)
        learning_rate /= 10**(1/7)
        optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate, 
                                     betas=(0.9, 0.999), eps=1e-04)
        with open('/content/drive/My Drive/IWAE_VAE/outfile_stage1_mod'+stage1_mod+'stage2_mod'+stage2_mod+'_layer'+str(layer)+'_k'+str(k)+'_'+'.txt', 'a') as outfile:
            if layer == 1:
                nll, A_u = test_function_1(net)
            else:
                nll, A_u = test_function_2(net)
            print('NLL:', nll, 'active units:', A_u)
            outfile.write('test average (NLL):'+str(nll)+'\n')
            outfile.write('test average (active units):'+str(A_u)+'\n')
            outfile.write('learning rate decay'+'\n')
        print("learning rate=", learning_rate)
    running_loss = 0.0
    for i,data in enumerate(trainloader, 0):
        train = data.view(-1,1,28*28).to(device)
        optimizer.zero_grad()
        
        output = net(train.float())
        log_prob_condi = torch.sum(output[0]*train.repeat(1,k,1)-torch.log(1+torch.exp(output[0])), 2)

        if layer == 1:
            # stochastic layer
            h = (output[1].repeat(1,k,1) + torch.exp(output[2].repeat(1,k,1))*output[3])
            # log weights, unnormalized
            log_weights = log_prob_condi-(h*h).sum(2)/2+(output[3]*output[3]).sum(2)/2+output[2].repeat(1,k,1).sum(2)
        else:
            # stochastic layer h1 minus mu3
            h1 = output[1].repeat(1,k,1) + torch.exp(output[4].repeat(1,k,1))*output[7]-output[3]
            # stochastic layer h2
            h2 = (output[2]+torch.exp(output[5])*output[8])
            # log weights, unnormalized
            log_p_h1_h2 = -(h1*h1/torch.exp(2*output[6])).sum(2)/2-output[6].sum(2)
            log_q_h1_x = -(output[7]*output[7]).sum(2)/2-output[4].repeat(1,k,1).sum(2)
            log_q_h2_h1 = -(output[8]*output[8]).sum(2)/2-output[5].sum(2)
            log_weights = log_prob_condi+log_p_h1_h2-(h2*h2).sum(2)/2-log_q_h1_x-log_q_h2_h1
        
        log_weights.to(device)
        if stage2_mod == 'vae':
            loss = -log_weights.mean().to(device)
        else:
            # sample one index from k sets of hidden values
            temp = torch.exp(F.log_softmax(log_weights-log_weights.min(1)[0].view(-1,1),1)).to(device)
            temp1 = torch.multinomial(temp,1).flatten().to(device)+index_vec
            # estimate loss
            loss = -torch.take(log_weights, temp1).mean().to(device)
        
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i%500 == 499:
            print('[%d, %5d] loss: %.3f' %
                 (epoch+1, i + 1, running_loss/500))
            with open('/content/drive/My Drive/IWAE_VAE/outfile_stage1_mod'+stage1_mod+'stage2_mod'+stage2_mod+'_layer'+str(layer)+'_k'+str(k)+'_'+'.txt', 'a') as outfile:
                outfile.write('[%d, %5d] loss: %.3f' %
                 (epoch+1, i + 1, running_loss/500)+'\n')
            running_loss = 0.0
        
PATH = '/content/drive/My Drive/IWAE_VAE/model/stage1_mod'+stage1_mod+'stage2_mod'+stage2_mod+'_net'+'_layer'+str(layer)+'_k'+str(k)+'_'+'.pth'
torch.save(net.state_dict(), PATH)

print('Finished Training')
with open('/content/drive/My Drive/IWAE_VAE/outfile_stage1_mod'+stage1_mod+'stage2_mod'+stage2_mod+'_layer'+str(layer)+'_k'+str(k)+'_'+'.txt', 'a') as outfile:
    outfile.write('Finished Training'+'\n'+'time cost:'+str(time.time()-start)+'\n')

